# Test title

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [4]:
rt_sched = query_sql(
    """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_sched_routes`
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
rt_sched['service_date'] = pd.to_datetime(rt_sched['service_date'])
rt_sched['weekday'] = pd.Series(rt_sched.service_date).dt.day_name()
rt_sched['month'] =  pd.Series(rt_sched.service_date).dt.month_name()

In [6]:
rt_sched.sample(5)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
2011,300,Big Blue Bus,0,3501,44,2022-05-02,104,0,0.00,Monday,May
34,300,Big Blue Bus,0,3354,2,2022-03-11,82,71,0.87,Friday,March
1118,300,Big Blue Bus,0,3488,R10,2022-06-06,6,4,0.67,Monday,June
1055,300,Big Blue Bus,0,3487,9,2022-06-28,70,69,0.99,Tuesday,June
1788,300,Big Blue Bus,0,3496,R7,2022-04-05,83,78,0.94,Tuesday,April


In [8]:
# (rt_sched
#  >>group_by(_.calitp_itp_id,
#             _.agency_name,
#             _.calitp_url_number,
#             _.service_date,
#             _.weekday,
#            _.month
#            )
#  >>summarize(total_num_sched = _.num_sched.sum() ,
#              total_num_vp = _.num_vp.sum())
#  >>mutate(pct_w_vp = (_.total_num_vp)/(_.total_num_sched))
#             )


In [9]:
# def by_date(df, sum1_sched, sum2_vp):
#     agg_df = (df
#      >>group_by(_.calitp_itp_id,
#                 _.agency_name,
#                 _.calitp_url_number,
#                 _.service_date,
#                 _.weekday,
#                _.month)
#      >>summarize(total_num_sched = (_[sum1_sched].sum()),
#              total_num_vp = (_[sum2_vp].sum()))
#      >>mutate(pct_w_rt = (_.total_num_vp)/(_.total_num_sched))
#             )
#     return agg_df


In [10]:
# cols_to_agg = ['calitp_itp_id', 'calitp_url_number','service_date', 'weekday', 'month']

In [11]:
# rt_sched_agg = by_date(rt_sched, 'num_sched', 'num_vp')

In [12]:
# rt_sched_agg

In [13]:
# check to see if the function in utils works the same

In [7]:
rt_sched_agg = utils.agg_by_date(rt_sched,'num_sched', 'num_vp')

In [8]:
rt_sched_agg

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,300,Big Blue Bus,0,2022-03-01,Tuesday,March,1690,1477,0.87
1,300,Big Blue Bus,0,2022-03-02,Wednesday,March,1704,1585,0.93
2,300,Big Blue Bus,0,2022-03-03,Thursday,March,1706,1576,0.92
3,300,Big Blue Bus,0,2022-03-04,Friday,March,1601,1487,0.93
4,300,Big Blue Bus,0,2022-03-05,Saturday,March,873,576,0.66
...,...,...,...,...,...,...,...,...,...
117,300,Big Blue Bus,0,2022-06-26,Sunday,June,772,500,0.65
118,300,Big Blue Bus,0,2022-06-27,Monday,June,1465,1017,0.69
119,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1559,1457,0.93
120,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [9]:
## Looking at Routes

In [10]:
rt_sched>>count(_.route_id)>>arrange(-_.n)

,route_id,n
20,3479,96
21,3480,96
22,3481,96
24,3485,96
25,3486,96
26,3487,96
28,3489,96
29,3490,96
32,3493,96
33,3494,96


In [11]:
rt_sched>>filter(_.service_date=='2022-04-01')>>summarize(n_routes = _.route_id.nunique())

,n_routes
0,19


In [12]:
(rt_sched>>group_by(_.service_date)>>summarize(n_routes=_.route_id.nunique())>>arrange(-_.n_routes)).n_routes.describe()

count   122.00
mean     16.63
std       3.95
min      10.00
25%      11.00
50%      19.00
75%      19.00
max      20.00
Name: n_routes, dtype: float64

In [13]:
#checking one route
(rt_sched>>filter(_.route_id=='3488')
).head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
1096,300,Big Blue Bus,0,3488,R10,2022-05-10,6,6,1.00,Tuesday,May
1097,300,Big Blue Bus,0,3488,R10,2022-04-28,6,5,0.83,Thursday,April
1098,300,Big Blue Bus,0,3488,R10,2022-05-23,6,4,0.67,Monday,May
1099,300,Big Blue Bus,0,3488,R10,2022-04-27,6,6,1.00,Wednesday,April
1100,300,Big Blue Bus,0,3488,R10,2022-06-07,6,6,1.00,Tuesday,June


In [34]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['route_short_name', "pct_w_vp"], nearest=True)
selection = alt.selection_multi(
                          fields=['route_short_name'], bind='legend')

line = alt.Chart(rt_sched).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=('Percent with Vehicle Positions')),
    color='route_short_name',
    strokeDash='route_short_name',
      tooltip = alt.Tooltip(["route_short_name", "service_date","pct_w_vp"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
                size=alt.condition(~highlight, alt.value(2), alt.value(5))
             ).properties(title={
                 "text": ["Average Percent of Scheduled Trips with Vehicle Postions Data", "by Route"],
             }).add_selection(
                 selection, highlight).properties(width=600)
line_chart = styleguide.preset_chart_config(line)
line_chart = _dla_utils.add_tooltip(line_chart, 'route_short_name', 'pct_w_vp')
line_chart.properties(width=600)

alt.Chart(...)

In [35]:
rt_sched>>filter(_.pct_w_vp<.5)>>count(_.route_id)

,route_id,n
0,3357,1
1,3361,3
2,3365,2
3,3371,19
4,3374,1
5,3489,1
6,3491,1
7,3501,55


* something funky happening with route 3501. 
* checking trip updates

In [36]:
(rt_sched>>filter(_.route_id=='3501')>>arrange(_.service_date))

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
1998,300,Big Blue Bus,0,3501,44,2022-03-28,104,88,0.85,Monday,March
2028,300,Big Blue Bus,0,3501,44,2022-03-29,107,95,0.89,Tuesday,March
2026,300,Big Blue Bus,0,3501,44,2022-03-30,107,107,1.00,Wednesday,March
2027,300,Big Blue Bus,0,3501,44,2022-03-31,107,107,1.00,Thursday,March
1986,300,Big Blue Bus,0,3501,44,2022-04-01,54,51,0.94,Friday,April
2015,300,Big Blue Bus,0,3501,44,2022-04-04,104,0,0.00,Monday,April
2007,300,Big Blue Bus,0,3501,44,2022-04-05,104,0,0.00,Tuesday,April
2025,300,Big Blue Bus,0,3501,44,2022-04-06,104,0,0.00,Wednesday,April
2009,300,Big Blue Bus,0,3501,44,2022-04-07,104,0,0.00,Thursday,April
1984,300,Big Blue Bus,0,3501,44,2022-04-08,52,0,0.00,Friday,April


In [37]:

## seeing what the pct_with_rt would look like without route 3501

In [38]:
no_3501 = utils.agg_by_date((rt_sched>>filter(_.route_id != '3501')),'num_sched', 'num_vp')

In [39]:
no_3501.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
96,300,Big Blue Bus,0,2022-06-05,Sunday,June,773,557,0.72


In [40]:
with_3501 = utils.agg_by_date((rt_sched),'num_sched', 'num_vp')
with_3501.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
118,300,Big Blue Bus,0,2022-06-27,Monday,June,1465,1017,0.69


In [41]:

line_1 = (alt.Chart(with_3501).mark_line(color='navy').encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    opacity = alt.Opacity('with_3501:Q', legend=alt.Legend(title="With 3501"))
))
line_2 = alt.Chart(no_3501).mark_line(color='orange').encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    opacity= alt.Opacity('no_3501:Q', legend=alt.Legend(title="Without 3501")))
chart = line_1+line_2
chart
# line_chart = styleguide.preset_chart_config(chart)
# line_chart = _dla_utils.add_tooltip(line_chart, 'service_date:O', 'pct_w_vp:Q')
# line_chart.properties(width=600)


alt.LayerChart(...)

In [42]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
936,300,Big Blue Bus,0,3486,8,2022-05-16,71,47,0.66,Monday,May


In [43]:
utils.groupby_onecol(rt_sched, 'month', 'pct_w_vp')

,month,avg
2,March,0.84
0,April,0.86
3,May,0.85
1,June,0.86


In [44]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
1717,300,Big Blue Bus,0,3495,R3,2022-06-14,59,59,1.00,Tuesday,June


In [45]:
route_weekday = (utils.groupby_twocol(rt_sched, 'route_short_name', 'weekday', 'pct_w_vp', 'weekday'))

In [46]:
route_weekday.sample()

,route_short_name,weekday,avg
119,R7,Tuesday,0.93


In [47]:
cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [49]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['route_short_name', "avg"], nearest=True)
selection = alt.selection_multi(
                          fields=['route_short_name'], bind='legend')

line = alt.Chart(route_weekday).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'),  sort=cats_day),
    y=alt.Y('avg:Q', title=('Average Percent with Vehicle Positions')),
    color='route_short_name',
  #  strokeDash='route_short_name',
    tooltip = alt.Tooltip(["route_short_name", "weekday","avg"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
                size=alt.condition(~highlight, alt.value(2), alt.value(5))
             ).properties(title={
                 "text": ["Average Percent of Scheduled Trips with Vehicle Postions Data", "by Route"],
             }).add_selection(
                 selection, highlight).properties(width=600)
line
#line_chart = styleguide.preset_chart_config(line)

alt.Chart(...)